In [1]:
import logging
from io import StringIO

import pandas as pd
import requests

In [ ]:
# variables which can be changed
GCP_PROJECT='adventures-on-gcp'
BQ_DATASET='covid_stats'
BQ_TABLE='world_data'
date='2021-04-20'

In [ ]:
logging.debug('downloading data')
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
r = requests.get(url)

f = StringIO(r.text)
f.seek(0)
df = pd.read_csv(f)
f.close()

In [ ]:
df_latest = df[df.date==date]

In [ ]:
df_world = df_latest[df_latest['location'] == 'World']

In [ ]:
from google.cloud import bigquery
bq = bigquery.Client(project=GCP_PROJECT)


In [ ]:
logging.debug('uploading to BQ')
job_config = bigquery.LoadJobConfig()

table_id = f'{BQ_DATASET}.{BQ_TABLE}'
job = bigquery.LoadJobConfig()
job.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
job.create_disposition = bigquery.CreateDisposition.CREATE_IF_NEEDED

job = bq.load_table_from_dataframe(df_world, table_id, job_config=job_config)

job.result()
logging.info('completed')